<div style="text-align: center; border: 1px solid black; padding: 10px;">
  <span style="color: blue; font-size: 45px;">
    IBM Employee Churn Prediction & Analysis
  </span>
</div>


# <span style="color:green; text-align: right;">CodeUp</span>


### By: Miatta Sinayoko
### Quintela Cohort: Data Science



 <span style="font-size: 40px;">EXPLORATORY DATA ANALYSIS</span>

# ACQUIRE

In [1]:
# Import Libraries
import pandas as pd
import numpy as np
import os
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats as stats

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from IPython.display import display, display_html
from sklearn.dummy import DummyClassifier

In [2]:
# Create function to retrieve HR_churn data
def get_HR_churn_data(file_path):
    """
    Load the dataset from the specified CSV file path.
    
    Parameters:
        file_path (str): The full file path to the CSV dataset file.
        
    Returns:
        pandas.DataFrame: The loaded dataset as a DataFrame.
    """
    try:
        # Load the dataset using pandas read_csv function
        df = pd.read_csv(file_path)
        print("Dataset successfully loaded.")
        return df
    except FileNotFoundError:
        print(f"Error: File '{file_path}' not found.")
        return None
    except Exception as e:
        print(f"Error occurred while loading the dataset: {e}")
        return None


In [4]:
# Replace 'path/to/your/Data_Hr_Analytics_S3.csv' with the actual full file path to your dataset file
file_path = '/Users/miattas/codeup-data-science/Employee-Churn-Prediction/Data_Hr_Analytics_S3.csv'

# Call the function to load the dataset
HR_df = get_HR_churn_data(file_path)


Dataset successfully loaded.


In [5]:
# To inspect the first few rows of the DataFrame, you can use the head functiotelco.head()
HR_df.head().T

   

,0,1,2,3,4
Age,41,49,37,33,27
Attrition,Yes,No,Yes,No,No
BusinessTravel,Travel_Rarely,Travel_Frequently,Travel_Rarely,Travel_Frequently,Travel_Rarely
DailyRate,1102,279,1373,1392,591
Department,Sales,Research & Development,Research & Development,Research & Development,Research & Development
DistanceFromHome,1,8,2,3,2
Education,2,1,2,4,1
EducationField,Life Sciences,Life Sciences,Other,Life Sciences,Medical
EmployeeCount,1,1,1,1,1
EmployeeNumber,1,2,4,5,7


**Data includes both numerical and categorical columns containing personal and employment details.**

In [6]:
HR_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1470 entries, 0 to 1469
Data columns (total 35 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Age                       1470 non-null   int64 
 1   Attrition                 1470 non-null   object
 2   BusinessTravel            1470 non-null   object
 3   DailyRate                 1470 non-null   int64 
 4   Department                1470 non-null   object
 5   DistanceFromHome          1470 non-null   int64 
 6   Education                 1470 non-null   int64 
 7   EducationField            1470 non-null   object
 8   EmployeeCount             1470 non-null   int64 
 9   EmployeeNumber            1470 non-null   int64 
 10  EnvironmentSatisfaction   1470 non-null   int64 
 11  Gender                    1470 non-null   object
 12  HourlyRate                1470 non-null   int64 
 13  JobInvolvement            1470 non-null   int64 
 14  JobLevel                

**Data contains no missing values**

### OBSERVED NUMERICAL FEATURES


In [7]:
HR_df.describe()

,Age,DailyRate,DistanceFromHome,Education,EmployeeCount,EmployeeNumber,EnvironmentSatisfaction,HourlyRate,JobInvolvement,JobLevel,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
count,1470.000000,1470.000000,1470.000000,1470.000000,1470.0,1470.000000,1470.000000,1470.000000,1470.000000,1470.000000,...,1470.000000,1470.0,1470.000000,1470.000000,1470.000000,1470.000000,1470.000000,1470.000000,1470.000000,1470.000000
mean,36.923810,802.485714,9.192517,2.912925,1.0,1024.865306,2.721769,65.891156,2.729932,2.063946,...,2.712245,80.0,0.793878,11.279592,2.799320,2.761224,7.008163,4.229252,2.187755,4.123129
std,9.135373,403.509100,8.106864,1.024165,0.0,602.024335,1.093082,20.329428,0.711561,1.106940,...,1.081209,0.0,0.852077,7.780782,1.289271,0.706476,6.126525,3.623137,3.222430,3.568136
min,18.000000,102.000000,1.000000,1.000000,1.0,1.000000,1.000000,30.000000,1.000000,1.000000,...,1.000000,80.0,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000
25%,30.000000,465.000000,2.000000,2.000000,1.0,491.250000,2.000000,48.000000,2.000000,1.000000,...,2.000000,80.0,0.000000,6.000000,2.000000,2.000000,3.000000,2.000000,0.000000,2.000000
50%,36.000000,802.000000,7.000000,3.000000,1.0,1020.500000,3.000000,66.000000,3.000000,2.000000,...,3.000000,80.0,1.000000,10.000000,3.000000,3.000000,5.000000,3.000000,1.000000,3.000000
75%,43.000000,1157.000000,14.000000,4.000000,1.0,1555.750000,4.000000,83.750000,3.000000,3.000000,...,4.000000,80.0,1.000000,15.000000,3.000000,3.000000,9.000000,7.000000,3.000000,7.000000
max,60.000000,1499.000000,29.000000,5.000000,1.0,2068.000000,4.000000,100.000000,4.000000,5.000000,...,4.000000,80.0,3.000000,40.000000,6.000000,4.000000,40.000000,18.000000,15.000000,17.000000


In [8]:
HR_df.shape

(1470, 35)

In [9]:
HR_df.isnull().sum()

Age                         0
Attrition                   0
BusinessTravel              0
DailyRate                   0
Department                  0
DistanceFromHome            0
Education                   0
EducationField              0
EmployeeCount               0
EmployeeNumber              0
EnvironmentSatisfaction     0
Gender                      0
HourlyRate                  0
JobInvolvement              0
JobLevel                    0
JobRole                     0
JobSatisfaction             0
MaritalStatus               0
MonthlyIncome               0
MonthlyRate                 0
NumCompaniesWorked          0
Over18                      0
OverTime                    0
PercentSalaryHike           0
PerformanceRating           0
RelationshipSatisfaction    0
StandardHours               0
StockOptionLevel            0
TotalWorkingYears           0
TrainingTimesLastYear       0
WorkLifeBalance             0
YearsAtCompany              0
YearsInCurrentRole          0
YearsSince

In [10]:
HR_df.sample(5)

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
363,33,Yes,Travel_Rarely,350,Sales,5,3,Marketing,1,485,...,2,80,0,1,2,3,1,0,0,0
1455,40,No,Travel_Rarely,1322,Research & Development,2,4,Life Sciences,1,2048,...,4,80,0,8,2,3,2,2,2,2
1467,27,No,Travel_Rarely,155,Research & Development,4,3,Life Sciences,1,2064,...,2,80,1,6,0,3,6,2,0,3
1072,29,No,Travel_Frequently,410,Research & Development,2,1,Life Sciences,1,1513,...,3,80,3,4,3,3,3,2,0,2
109,22,No,Travel_Rarely,534,Research & Development,15,3,Medical,1,144,...,3,80,0,1,5,3,0,0,0,0


# Acquire Summary
* Data acquired from file path via (https://ieee-dataport.org/documents/ibm-hr-analytics-employee-attrition-performance) 
* The data set has 35 columns and 1,470 rows 
* Each row is a single property
* Each column contains information about the properties




In [ ]:
HR_df=HR_dfset_index('Date').sort_index()
HR_df.head()

# PREPARE

In [ ]:
def prep_HR_data(df):
    '''
    This function takes in the DataFrame from get_telco_churn_data
    and returns the DataFrame with preprocessing applied 
    '''
    # Drop duplicated columns and rows
    df = df.loc[:, ~df.columns.duplicated()]
    df.drop_duplicates(inplace=True)

    # Replace ' ' in 'total_charges' column with '0' and change its data type to float
    df.total_charges = df.total_charges.replace(' ', '0').astype(float)

    # Change data type for boolean columns
    for col in ['churn', 'partner', 'dependents', 'phone_service']:
        df[col] = df[col].map({'No': 0, 'Yes': 1})

    return df

In [ ]:
def split_data(df):
    '''
    This function takes in a DataFrame and returns train, validate, and test DataFrames.
    '''
    # Create train_validate and test datasets
    train_validate, test = train_test_split(df, test_size=.2, random_state=123, stratify=df.churn)

    # Split train_validate into train and validate datasets
    train, validate = train_test_split(train_validate, test_size=.3, random_state=123, stratify=train_validate.churn)

    return train, validate, test

# EXPLORE

In [ ]:
# Change the name of the data to 'HR'
HR = prep_HR_churn(HR)

In [ ]:
prep_HR_churn(HR)


In [ ]:
# Review the data
HR.head()

In [ ]:
#The data types in this dataset are integers and object.
HR.info()

### DATA TYPE SUMMARY
- 26 integer data types
- 9 object data type
- null values



In [ ]:
# Summary of the statistical properties of HR dataset
HR.describe()

### Further exploretion for valuable insights.

In [ ]:
#We need to understand why customers are churning
HR.churn.value_counts()

# 26.54% (1 in 3)of customers churn, totaling 1869 out of 7043 

In [ ]:
# Determine how many are senior citizens and non-senior citizens 

HR.Age.value_counts()

# 16.21% (1 in 6) of customers are seniors (1142 out of 7043)

In [ ]:
# Average tenure, max tenure, min tenure
HR.tenure.mean(), HR.tenure.max(), HR.tenure.min()

In [ ]:
# Average monthly charges, max monthly charges, min monthly charges
HR.monthly_charges.mean(), HR.monthly_charges.max(), HR.monthly_charges.min()

In [ ]:
# Group by churn and Age
# Senior_churn= telco.groupby(['churn','Age'])


In [ ]:
# Value counts for churn as int not obj
(HR.churn == 'Yes').astype(int).value_counts()

In [ ]:
HR.head()


In [ ]:
HR.head()


In [ ]:
# Show all column names for next steps of value count loop
HR.columns

In [ ]:
# List the columns that will be used in the loop.

columns = ['gender', 'senior_citizen', 'partner', 'dependents', 'tenure',
       'phone_service', 'multiple_lines', 'internet_service_type_id',
       'online_security', 'online_backup', 'device_protection', 'tech_support',
       'streaming_tv', 'streaming_movies', 'contract_type_id',
       'paperless_billing', 'payment_type_id', 'monthly_charges',
       'total_charges', 'churn', 'contract_type_id', 'contract_type',
       'payment_type_id', 'payment_type', 'internet_service_type_id',
       'internet_service_type']

In [ ]:
# Create for loop print value counts and percents
for col in columns:
    print(col)
    print(HR[col].value_counts())
    print('----------------------------------------')
    print(HR[col].value_counts(normalize=True))
    print('~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~')
    print('~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~')

# Data Prep

In [ ]:
import warnings
warnings.filterwarnings('ignore', category=FutureWarning)

In [ ]:
# Identify a  correlation, age appears to correlate with churn the most
HR = HR.corr()
HR
# The highest positive correlation with churn are Age and monthly_charges 


In [ ]:
# Provides correlation with JUST churn
HR = HR.correlation['churn']
HR_corr_churn

# senior_citizen is still the highest correlation
#This data will help us pinpoint why customers churn

In [ ]:
# Plot displays Age (1) vs non-senior citizen (0)
# Plot displays churn (1) vs do not churn (0)
sns.countplot(x='churn', hue='Age', data=telco)

In [ ]:
# Heatmap viual to show data correlation 
plt.figure(figsize=(16,9))

sns.heatmap(telco.corr(), cmap='YlOrRd', center=0, annot=True)

plt.title('Correlation of HR Data')

plt.show()

# MODELING & TESTING 

### Finding the best model
- churn and Age are both categorical
- 2 discrete variables
- use chi2 testing

In [ ]:
# Create confusion matrix
observed = pd.crosstab(HR.churn, HR.Age)
observed

Hypothosis:

**$H_{0}$**: there is no relationship between churn and employee age

**$H_{A}$**: there is a relationshp between churn and employee age

True Positive: Correctly predict there is a relationship and there is a relationship

True Negative: Correctly Predict there is no relationship and there is not relationship

False Positive: Incorectly predict there is a relationship and there is no relationship

False Negative: Incorrectly predict there is no relationship and there is a relationship


In [ ]:
# Define alpha
alpha = 0.05

In [ ]:
# chi2 contingency returns 4 different values
chi2, p, degf, expected = stats.chi2_contingency(observed)
chi2, p, degf, expected

In [ ]:
# Streamline data
print('Observed\n')
print(observed.values)
print('---------------------\nExpected\n')
print(expected.astype(int))
print('---------------------\n')
print(f'chi^2 = {chi2:.4f}')
print(f'p     = {p:.4f}')

In [ ]:
if p< alpha:
    print('We reject the null hypothesis')
else:
    print('We fail to reject the null hypothesis')

In [ ]:
We reject the null hypothesis

In [ ]:
We reject 

In [ ]:
. We believe that there is a relationship between churn and customer age

# TRAIN SPLIT

In [ ]:
import prepare
import acquire

In [ ]:
# Create train,validate and test datasets

train, test = train_test_split(telco, test_size=.2, random_state=123)
train, validate = train_test_split(train, test_size=.3, random_state=123)

In [ ]:
# Explore and plot with only train set


In [ ]:
# Crosstab, confusion matrix


In [ ]:
pd.crosstab(train.churn, train.model5)


In [ ]:
# make baseline


In [ ]:
X_col= ['senior_citizen','tenure', 'internet_service_type_id', 'contract_type_id']
y_col= 'churn'


In [ ]:
X_train = train[X_col]
y_train= train[y_col]

X_validate = validate[X_col]
y_validate= validate[y_col]

X_test = test[X_col]
y_test= test[y_col]